In [3]:
%pip install torch
%pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 2.6 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 3.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 3.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explici

In [1]:
import torch
from transformers import BertTokenizer, BertLMHeadModel

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertLMHeadModel.from_pretrained(model_name)

# Tokenize the input information
def tokenize_input(input_text):
    encoded_inputs = tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,
        max_length=512,  # Adjust the maximum sequence length as needed
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return encoded_inputs['input_ids'], encoded_inputs['attention_mask']

# Train the model
def train_model(train_inputs, train_masks, train_labels):
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    loss_fn = torch.nn.CrossEntropyLoss()
    num_epochs = 10
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(input_ids=train_inputs, attention_mask=train_masks, labels=train_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


# Example usage
import pandas as pd

data = pd.read_csv("Resume.csv")
df = pd.DataFrame(data, columns=["Resume_str", "Category"])

# Prepare inputs and labels
inputs = []
masks = []
labels = []
for _, row in df.iterrows():
    input_ids, attention_mask = tokenize_input(row['Category'])
    inputs.append(input_ids)
    masks.append(attention_mask)
    labels.append(tokenizer.encode_plus(row['Resume_str'], padding='max_length', truncation=True, return_tensors='pt', max_length=512)['input_ids'])

# Convert lists to tensors
inputs = torch.cat(inputs, dim=0)
masks = torch.cat(masks, dim=0)
labels = torch.cat(labels, dim=0)

# Split your data into training and validation sets
train_ratio = 0.8  # 80% for training, 20% for validation
train_size = int(train_ratio * len(inputs))

train_inputs = inputs[:train_size]
train_masks = masks[:train_size]
train_labels = labels[:train_size]

val_inputs = inputs[train_size:]
val_masks = masks[train_size:]
val_labels = labels[train_size:]

train_model(train_inputs, train_masks, train_labels)

# Generate resumes
def generate_resume(input_text):
    model.eval()
    input_ids, attention_mask = tokenize_input(input_text)
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=512)
    generated_resume = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_resume

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


: 

: 